In [1]:
!pip install tinyec

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import secrets
from tinyec import registry

In [3]:
def compress(pubKey):
    return hex(pubKey.x)

In [4]:
curve = registry.get_curve('secp256r1')

In [5]:
alicePrivKey = secrets.randbelow(curve.field.n)
alicePubKey = alicePrivKey * curve.g
print("Alice public key:", compress(alicePubKey))

Alice public key: 0x390ed99205010b90e7f9ef521c54ab5ab406301e2101c65ec622353815dfc5d6


In [6]:
bobPrivKey = secrets.randbelow(curve.field.n)
bobPubKey = bobPrivKey * curve.g
print("Bob public key:", compress(bobPubKey))

Bob public key: 0x21399815dde1d48f2800e8a4cf159d82523b54edc5ec6c020675adea3e262bb8


# ECDH

In [7]:
aliceSharedKey = alicePrivKey * bobPubKey
print("Alice shared key:", compress(aliceSharedKey))

Alice shared key: 0xdf608ea91ad785ee63625f29c5af6a5268a0cbd931f5eb725c67f8c98074fdf7


In [8]:
bobSharedKey = bobPrivKey * alicePubKey
print("Bob shared key:", compress(bobSharedKey))

Bob shared key: 0xdf608ea91ad785ee63625f29c5af6a5268a0cbd931f5eb725c67f8c98074fdf7


In [9]:
print("Equal shared keys:", aliceSharedKey == bobSharedKey)

Equal shared keys: True


# Selected

In [10]:
aliceSessionSecret = secrets.randbelow(curve.field.n)
print("aliceSessionSecret:", aliceSessionSecret)

aliceSelfPartialShared = aliceSessionSecret * curve.g
print("aliceSelfPartialShared:", compress(aliceSelfPartialShared))

toBob = (alicePrivKey + aliceSessionSecret) * bobPubKey
print("to Bob:", compress(toBob))

aliceSessionSecret: 110597949588124173913380997439899016580151652415201875188732749955098238273705
aliceSelfPartialShared: 0x373dd8bbb8f9b14d867fc01f160a8d0ad9b9b37274c73c93bad06c675b01e975
to Bob: 0xdf08262a26ce39880a6217685dd27bc60dbdebd9c2b444fd7b41b3f80c80bd14


In [11]:
bobSessionSecret = secrets.randbelow(curve.field.n)
print("bobSessionSecret:", bobSessionSecret)

bobSelfPartialShared = bobSessionSecret * curve.g
print("bobSelfPartialShared:", compress(bobSelfPartialShared))

toAlice = (bobPrivKey + bobSessionSecret) * alicePubKey
print("to Alice:", compress(toAlice))

bobSessionSecret: 105681370464123269745508846097648435958747218225751481596408454102168536662430
bobSelfPartialShared: 0xe186dd6fa64f9a6f825a279952b3280c3910dd86349359496b0a213d3439b1ae
to Alice: 0x21b82cefc38ec78898122166713a2de8e8af6a65b6a6498525d07c9382079506


## Computing shared

In [15]:
alicePrivKeyInv = pow(alicePrivKey, -1, curve.field.n)

alicePartialShared =  (toAlice * alicePrivKeyInv) - bobPubKey
print("alice Partial Shared:", compress(alicePartialShared))

aliceShared = aliceSelfPartialShared + alicePartialShared
print("aliceShared:", compress(aliceShared))

alice Partial Shared: 0xe186dd6fa64f9a6f825a279952b3280c3910dd86349359496b0a213d3439b1ae
aliceShared: 0xe76884e0caf40edafe7aa7d7046c1d1b78f1041202a001035c607018dbda75e5


In [16]:
bobPrivKeyInv = pow(bobPrivKey, -1, curve.field.n)

bobPartialShared = (toBob * bobPrivKeyInv) - alicePubKey
print("bob Partial Shared:", compress(bobPartialShared))

bobShared = bobSelfPartialShared + bobPartialShared
print("bobShared:", compress(bobShared))

bob Partial Shared: 0x373dd8bbb8f9b14d867fc01f160a8d0ad9b9b37274c73c93bad06c675b01e975
bobShared: 0xe76884e0caf40edafe7aa7d7046c1d1b78f1041202a001035c607018dbda75e5


In [17]:
print("Equal shared keys:", aliceShared.x == bobShared.x)

Equal shared keys: True
